In [1]:
%matplotlib inline
import numpy as np

In [8]:
from p2ch10.datasets import getCandidateInfoList, getCt, LunaDataset, raw_cache

In [3]:
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=False)
positiveInfo_list = [x for x in candidateInfo_list if x[0]]
diameter_list = [x[1] for x in positiveInfo_list]

In [4]:
print(len(positiveInfo_list))
print(positiveInfo_list[0])


1351
CandidateInfoTuple(isNodule_bool=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(75.7212243102, 92.8873310394, -119.270564052))


In [5]:
sample_data = positiveInfo_list[0].series_uid
dataset = LunaDataset()
dataset[0]

2024-02-28 19:25:05,364 INFO     pid:50046 p2ch10.datasets:153:__init__ <p2ch10.datasets.LunaDataset object at 0x282067b50>: 551065 training samples


(tensor([[[[-880., -846., -825.,  ..., -812., -772., -723.],
           [-862., -805., -813.,  ..., -833., -829., -801.],
           [-795., -770., -819.,  ..., -830., -808., -804.],
           ...,
           [-859., -858., -882.,  ...,  -35.,   -4.,   11.],
           [-883., -878., -876.,  ...,    0.,  -13.,  -35.],
           [-878., -845., -816.,  ...,    2.,  -15.,  -38.]],
 
          [[-924., -914., -879.,  ..., -794., -732., -638.],
           [-897., -881., -858.,  ..., -821., -787., -741.],
           [-884., -877., -875.,  ..., -824., -808., -811.],
           ...,
           [-891., -868., -784.,  ...,  -80.,  -99., -109.],
           [-887., -856., -759.,  ...,  -69.,  -83.,  -84.],
           [-884., -835., -766.,  ..., -103., -102., -100.]],
 
          [[-843., -843., -857.,  ..., -846., -837., -857.],
           [-862., -854., -864.,  ..., -844., -821., -831.],
           [-861., -843., -831.,  ..., -815., -807., -816.],
           ...,
           [-857., -789., -707.

### Exercises

1. Implement a program that iterates through a LunaDataset instance, and time how long it takes to do so. <br> In the interest of time, it might make sense to have an option to limit the iterations to the first N=1000 samples.<br>
    1. How long does it take to run the first time?<br>
        Roughly 2 mins 43 seconds on M1 Max Macbook (did not use `mps` as device) <br><br>
    2. How long does it take to run the second time?<br>
        2 seconds! <br><br>
    3. What does clearing the cache do to the runtime?<br>
        Well, as expected it again takes too long! Roughly 2 mins 30 secs<br><br>
        
    4. What does using the last N=1000 samples do to the first/second runtime? <br>
        Strangely it took only 2 seconds even after clearning the cache. The second time was even shorter <br>
        Have to check this agian <br><br>
        

2. Change the LunaDataset implementation to randomize the sample list during `__init__`. <br> Clear the cache, and run the modified version. What does that do to the runtime of the first and second runs?

3. Revert the randomization, and comment out the @functools.lru_cache(1, typed=True) decorator to getCt. <br>Clear the cache, and run the modified version. How does the runtime change now?

In [13]:
%%time

for i, data in enumerate(dataset):
    if i >=1000:
        break

CPU times: user 1min 6s, sys: 33.2 s, total: 1min 39s
Wall time: 1min 51s


In [14]:
%%timeit

for i, data in enumerate(dataset):
    if i >=1000:
        break

231 ms ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
getCandidateInfoList.cache_clear()
getCt.cache_clear()
raw_cache.clear()

1001

In [16]:
%%time

for i, data in enumerate(dataset):
    if i >=1000:
        break

CPU times: user 1min 31s, sys: 45.5 s, total: 2min 17s
Wall time: 2min 33s


##### Last 1000 samples

In [25]:
%%time
num_samples = len(dataset)
start_idx = num_samples - 1000

for idx in range(start_idx, num_samples):
    temp = dataset[idx]


CPU times: user 98.5 ms, sys: 76.8 ms, total: 175 ms
Wall time: 425 ms


In [26]:
getCandidateInfoList.cache_clear()
getCt.cache_clear()
raw_cache.clear()

2001

In [27]:
%%time
num_samples = len(dataset)
start_idx = num_samples - 1000

for idx in range(start_idx, num_samples):
    temp = dataset[idx]

CPU times: user 1.13 s, sys: 819 ms, total: 1.95 s
Wall time: 2.57 s


In [28]:
%%timeit
num_samples = len(dataset)
start_idx = num_samples - 1000

for idx in range(start_idx, num_samples):
    temp = dataset[idx]

265 ms ± 78.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Randomize